*An intro to high performance custom arrays by Matt Bauman*

https://youtu.be/jS9eouMJf_Y

In [44]:
using BenchmarkTools

In [45]:
A = rand(1000,1000)

1000×1000 Array{Float64,2}:
 0.801982   0.243768   0.341278    …  0.837879   0.263734   0.446793 
 0.452003   0.641526   0.0363849      0.626592   0.324803   0.14585  
 0.994354   0.837405   0.756541       0.684074   0.823409   0.314149 
 0.841755   0.431586   0.912753       0.769504   0.12217    0.638213 
 0.367195   0.853167   0.54268        0.138248   0.547738   0.168017 
 0.867034   0.696406   0.299208    …  0.211183   0.225516   0.156253 
 0.490407   0.176928   0.818781       0.364192   0.217029   0.840258 
 0.0960673  0.185475   0.967371       0.331174   0.577374   0.580567 
 0.443353   0.10112    0.539404       0.480087   0.202106   0.322965 
 0.561636   0.828645   0.55612        0.859837   0.975529   0.174393 
 0.296877   0.0928802  0.591909    …  0.367502   0.0307187  0.899187 
 0.210813   0.485484   0.397255       0.0620697  0.507131   0.517838 
 0.80914    0.526056   0.267201       0.748461   0.246082   0.561428 
 ⋮                                 ⋱                          

In [46]:
function weighted_sum( A, weights=ones(size(A)))
    r = zero(A[1])
    for i in eachindex(A, weights)
        r += A[i]*weights[i]
    end
    return r
end
@btime weighted_sum($A)

  2.325 ms (2 allocations: 7.63 MiB)


500215.87431490654

In [47]:
@btime sum($A)

  255.154 μs (0 allocations: 0 bytes)


500215.87431491725

In [48]:
module V1
struct OnesMatrix <: AbstractArray{Int, 2}
    m::Int
    n::Int
end
Base.size(o::OnesMatrix) = (o.m, o.n)
Base.getindex(o::OnesMatrix, i::Int, j::Int) = 1
end 

Main.V1

In [49]:
x = V1.OnesMatrix(1000,1000)

1000×1000 Main.V1.OnesMatrix:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  

In [50]:
function weighted_sum( A, weights=V1.OnesMatrix(size(A)...))
    r = zero(A[1])
    for i in eachindex(A, weights)
        r += A[i]*weights[i]
    end
    return r
end
@btime weighted_sum($A)

  1.733 ms (0 allocations: 0 bytes)


500215.87431490654

Add bounds checking

In [51]:
module V2
struct OnesMatrix <: AbstractArray{Int, 2}
    m::Int
    n::Int
end
Base.size(o::OnesMatrix) = (o.m, o.n)
function Base.getindex(o::OnesMatrix, i::Int, j::Int)
    checkbounds(o, i, j)
    1
end
end 

Main.V2

In [52]:
function weighted_sum( A, weights=V2.OnesMatrix(size(A)...))
    r = zero(A[1])
    @inbounds for i in eachindex(A, weights)
        r += A[i]*weights[i]
    end
    return r
end
@btime weighted_sum($A)

  1.958 ms (0 allocations: 0 bytes)


500215.87431490654

In [53]:
module V3
struct OnesMatrix <: AbstractArray{Int, 2}
    m::Int
    n::Int
end
Base.size(o::OnesMatrix) = (o.m, o.n)
@inline function Base.getindex(o::OnesMatrix, i::Int, j::Int)
    @boundscheck begin
        checkbounds(o, i, j)
    end
    1
end
end 

Main.V3

In [54]:
function weighted_sum( A, weights=V3.OnesMatrix(size(A)...))
    r = zero(A[1])
    @inbounds for i in eachindex(A, weights)
        r += A[i]*weights[i]
    end
    return r
end
@btime weighted_sum($A)

  1.679 ms (0 allocations: 0 bytes)


500215.87431490654

In [55]:
module V4
struct OnesMatrix <: AbstractArray{Int, 2}
    m::Int
    n::Int
end
Base.size(o::OnesMatrix) = (o.m, o.n)
Base.IndexStyle(::Type{OnesMatrix}) = IndexLinear()
@inline function Base.getindex(o::OnesMatrix, i::Int)
    @boundscheck begin
        checkbounds(o, i)
    end
    1
end
end 

Main.V4

In [58]:
function weighted_sum( A, weights=V4.OnesMatrix(size(A)...))
    r = zero(A[1])
    for i in eachindex(A, weights)
        r += A[i]*weights[i]
    end
    return r
end
@btime weighted_sum($A)

  776.404 μs (0 allocations: 0 bytes)


500215.87431490654